# Парсеры для создания датасетов

## Общие функции и импорт библиотек

Общие функции для работы с парсингом

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, parse_qs
import os
import time
import json
import re

def ensure_directory_exists(url, base_dir="pages"):
    """Создает директорию для сохранения страницы"""
    parsed_url = urlparse(url)
    path = os.path.join(base_dir, parsed_url.netloc, *parsed_url.path.split('/')[:-1])
    os.makedirs(path, exist_ok=True)
    return path

def get_filename_from_url(url):
    """Генерирует имя файла из URL с сохранением параметров"""
    parsed_url = urlparse(url)

    # Обрабатываем путь
    path_parts = [p for p in parsed_url.path.split('/') if p]
    filename = '_'.join(path_parts) or 'index'

    # Обрабатываем параметры запроса
    if parsed_url.query:
        # Удаляем параметры, которые не нужны (например, session ID)
        query_params = parse_qs(parsed_url.query)

        if query_params:
            # Сортируем параметры для единообразия
            sorted_params = sorted(query_params.items())
            params_str = '_'.join(f"{k}-{v}" for k, v in sorted_params)
            filename += f"_{params_str}"

    # Заменяем недопустимые символы в имени файла
    filename = re.sub(r'[^\w\-_.]', '_', filename)

    # Ограничиваем длину имени файла (например, 100 символов)
    filename = filename[:100]

    return f"{filename}.html"

def save_page(url, content, base_dir="pages"):
    """Сохраняет HTML-страницу"""
    path = ensure_directory_exists(url, base_dir)
    filename = os.path.join(path, get_filename_from_url(url))
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(content)
    return filename

def load_page(url, base_dir="pages"):
    """Загружает сохраненную HTML-страницу"""
    path = ensure_directory_exists(url, base_dir)
    filename = os.path.join(path, get_filename_from_url(url))
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            return f.read()
    return None

def save_results(data, filename="results.json"):
    """Сохраняет результаты в JSON файл"""
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def load_results(filename="results.json"):
    """Загружает результаты из JSON файла"""
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def download_image(url, product_id, session, save_dir, retry=3):
    """Скачивает и сохраняет изображение с возможностью повтора"""
    for attempt in range(retry):
        try:
            response = session.get(url, stream=True, timeout=10)
            response.raise_for_status()

            # Определяем расширение файла
            content_type = response.headers.get('content-type', '')
            extension = '.jpg'  # по умолчанию
            if 'webp' in content_type:
                extension = '.webp'
            elif 'png' in content_type:
                extension = '.png'
            elif 'gif' in content_type:
                extension = '.gif'

            filename = f"{product_id}{extension}"
            filepath = os.path.join(save_dir, filename)

            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)

            return filepath
        except Exception as e:
            if attempt == retry - 1:
                print(f"Ошибка при загрузке {url}: {e}")
                return None
            time.sleep(2)  # Задержка перед повторной попыткой

def save_dataframe(df, base_filename='embroteka.csv'):
    """Сохраняет DataFrame в различных форматах"""
    df.to_csv(base_filename, index=False, encoding='utf-8')
    print(f"Данные сохранены в CSV: {base_filename}")

Далее выходные данные некоторых ячеек были очищены так как они ухудшали читаемость.

## Парстинг сайта embroteka.ru

### Создание датасета

Парсинг каталога сайта embroteka - оттуда достаются категории и для каждой категории проводится итерация по страницам, откуда собираются данные о вышивках.

In [ ]:
def get_categories_from_html(html_content, base_url):
    soup = BeautifulSoup(html_content, 'html.parser')
    categories = []

    category_items = soup.select('.header__catalog-aside > .header__catalog-menu > li')

    for item in category_items:
        main_link = item.find('a', class_='header__catalog-link')
        if not main_link:
            continue

        category_name = main_link.find('span', class_='header__catalog-link-text').get_text(strip=True)
        category_url = urljoin(base_url, main_link['href'])
        categories.append({
            'name': category_name,
            'url': category_url
        })

    return categories

def get_product_data(soup, base_url):
    products = []
    product_items = soup.select('.products__item')

    for item in product_items:
        name = item.select_one('.products__item-title').get_text(strip=True)

        relative_url = item.select_one('.products__item-title')['href']
        url = urljoin(base_url, relative_url)

        price = item.select_one('.products__item-price')
        if price.select_one('del') is not None:
            price = price.select_one('del')
        price = price.get_text(strip=True).replace('₽', '').strip()

        imgs = item.select('.products__item-image img')
        img_urls = []
        for img in imgs:
            image_url = img.get('data-src')
            if not image_url:
                image_url = img.get('src')
            if image_url.startswith('data:image'):
                print("!!!!", img)
                continue
            img_urls.append(urljoin(base_url, image_url))

        products.append({
            'name': name,
            'url': url,
            'price': price,
            'image_url': img_urls,
        })

    return products

def get_all_products_from_category(category_url, session, use_cache=True):
    all_products = []
    page_num = 1

    while True:
        print(f'    Обрабатываю страницу: {page_num}')
        page_url = f"{category_url}/?limit=100&page={page_num}" if page_num > 1 else f"{category_url}/?limit=100"

        # Пробуем загрузить из кэша
        html_content = None
        if use_cache:
            html_content = load_page(page_url)

        # Если нет в кэше, скачиваем
        sleep = False
        if not html_content:
            sleep = True
            try:
                response = session.get(page_url)
                response.raise_for_status()
                html_content = response.text
                save_page(page_url, html_content)
            except Exception as e:
                print(f"Ошибка при загрузке страницы {page_url}: {e}")
                break

        soup = BeautifulSoup(html_content, 'html.parser')

        # Проверяем, есть ли товары на странице
        if not soup.select('.products__item'):
            break

        products = get_product_data(soup, category_url)
        all_products.extend(products)

        # Проверяем наличие кнопки "Следующая страница"
        next_page = soup.select_one('div.pagination__action a.ui-btn.show-more-prostore')
        if not next_page:
            break

        page_num += 1
        if sleep:
            time.sleep(1)  # Задержка между запросами

    return all_products

def parse_embroteka(base_url, use_cache=True):
    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    })

    try:
        # Пробуем загрузить главную страницу из кэша
        html_content = None
        if use_cache:
            html_content = load_page(base_url)

        # Если нет в кэше, скачиваем
        if not html_content:
            response = session.get(base_url)
            response.raise_for_status()
            html_content = response.text
            save_page(base_url, html_content)

        # Получаем список всех категорий
        categories = get_categories_from_html(html_content, base_url)
    except Exception as e:
        print(f"Ошибка при получении главной страницы: {e}")
        return None

    all_data = {}

    for category in categories:
        print(f"Обрабатываю категорию: {category['name']}")

        try:
            products = get_all_products_from_category(category['url'], session, use_cache)
            all_data[category['name']] = {
                'url': category['url'],
                'products': products
            }

            time.sleep(2)  # Задержка между категориями

        except Exception as e:
            print(f"Ошибка при обработке категории {category['name']}: {e}")
            continue

    return all_data

In [ ]:
base_url = "https://embroteka.ru"
use_cache = True  # Использовать сохраненные страницы если они есть

result = parse_embroteka(base_url, use_cache)

In [ ]:
if result:
    # Сохраняем результаты
    save_results(result, filename='embroteka.json')

    # Пример вывода результатов
    for category_name, data in result.items():
        print(f"\nКатегория: {category_name} ({len(data['products'])} товаров)")
        for product in data['products'][:3]:  # Выводим первые 3 товара для примера
            print(f"  - {product['name']} | Цена: {product['price']}")
else:
    print("Не удалось получить данные с сайта")


Категория: Астрология и Космос (183 товаров)
  - Baby Tom | Цена: 300
  - Behind | Цена: 6002
  - Eren x Nike logo | Цена: 200

Категория: Военные шевроны и нашивки (659 товаров)
  - Вежливые люди Ветераны СВО | Цена: 250
  - РСЗО Смерч | Цена: 150
  - Танк Т-72 | Цена: 150

Категория: Для детей (397 товаров)
  - Babyball | Цена: 379
  - Chucky \  Чакки | Цена: 500
  - Cow коровка | Цена: 200

Категория: Животные (1540 товаров)
  - 2 змеи | Цена: 650
  - Bear / Медведь | Цена: 199
  - Belissimo кот | Цена: 360

Категория: Кино, мульты, аниме, игры (2693 товаров)
  - 3 pochita | Цена: 200
  - 86 EIGHTY SIX/ Аниме Восемьдесят шесть | Цена: 300
  - Aang из аниме Аватар: легенда об Аанге | Цена: 250

Категория: Логотипы, гербы, шевроны (2105 товаров)
  - harley davidson харлей дэвидсон 3 размера | Цена: 120
  - Демон STATE OF NINJA из города XIII | Цена: 500
  - ХК Динамо Москва | Цена: 300

Категория: Люди (539 товаров)
  - 2pac | Цена: 300
  - 2pac / Tupac Shakur | Цена: 1600
  - 2pac/ 

### Заполнение описаний и деталей для каждого продукта

Так как из каталога можно достать не все данные, то далее производится итерация по всем страницам товаров и заполняется дополнительная информация о них.

In [ ]:
def parse_product_page(product_url, session, base_url, use_cache=True):
    """Парсит страницу продукта и извлекает характеристики и описание"""
    # Пробуем загрузить из кэша
    html_content = None
    if use_cache:
        html_content = load_page(product_url)

    # Если нет в кэше, скачиваем
    if not html_content:
        try:
            response = session.get(product_url)
            response.raise_for_status()
            html_content = response.text
            save_page(product_url, html_content)
        except Exception as e:
            print(f"Ошибка при загрузке страницы продукта {product_url}: {e}")
            return None

    soup = BeautifulSoup(html_content, 'html.parser')

    # Извлекаем характеристики
    characteristics = {}
    specs_table = soup.select_one('.details__specifications-table')
    if specs_table:
        rows = specs_table.select('tr')
        for row in rows:
            th = row.select_one('th')
            td = row.select_one('td')
            if th and td:
                key = th.get_text(strip=True)
                value = td.get_text(strip=True)
                characteristics[key] = value

    # Извлекаем описание
    description = ""
    description_section = soup.select_one('.details__accordion[data-accordion-content="description"]')
    if description_section:
        description = description_section.select_one('.editor').get_text(strip=True)

    # Извлекаем дополнительные изображения
    check_images = []
    gallery_items = soup.select('.sku__slide img')
    for img in gallery_items:
        image_url = img.get('data-src')
        if not image_url:
            image_url = img.get('src')
        if image_url.startswith('data:image'):
            continue
        check_images.append(urljoin(BASE_URL, image_url))

    return {
        'characteristics': characteristics,
        'description': description,
        'image_url': check_images,
    }

def enrich_product_data(product_data, session, base_url, use_cache=True):
    """Обогащает данные продукта информацией со страницы продукта"""
    print(f"Обрабатываю продукт: {product_data['name']}")

    product_info = parse_product_page(product_data['url'], session, base_url, use_cache)
    if product_info:
        product_data.update(product_info)
        product_data['downloaded'] = False

    # Добавляем задержку между запросами
    time.sleep(0.2)

    return product_data

def enrich_all_products_data(result, base_url, use_cache=True):
    """Обогащает данные всех продуктов в результате"""
    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    })

    for category_name, data in result.items():
        print(f"Обогащаю данные для категории: {category_name}")

        # Обрабатываем каждый продукт в категории
        for i, product in enumerate(data['products']):
            if 'characteristics' in product and 'description' in product:
                continue  # Пропускаем уже обработанные
            try:
                enriched_product = enrich_product_data(product, session, base_url, use_cache)
                data['products'][i] = enriched_product

                # Периодически сохраняем промежуточные результаты
                if i % 10 == 0:
                    save_results(result, "results_enriched.json")

            except Exception as e:
                print(f"Ошибка при обработке продукта {product['name']}: {e}")
                continue

    return result

In [ ]:
BASE_URL = "https://embroteka.ru"
SAVE_DIR = "embroteka_imgs"
os.makedirs(SAVE_DIR, exist_ok=True)

enriched_result = enrich_all_products_data(result, BASE_URL, use_cache=True)

save_results(enriched_result, "embroteka_imgs/embroteka.json")

### Скачивание картинок

Здесь по ранее найденным ссылкам скачаваются картинки.

In [ ]:
def download_images_for_products(result, save_dir):
    """Скачивает изображения для всех продуктов в датасете"""
    # Сессия для запросов
    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    })

    id = 0
    for category_name, data in result.items():
        for i, product in enumerate(data['products']):
            if 'downloaded' in product and product['downloaded']:
                id += len(product['local_image_path'])
                continue
            paths = []
            for img_url in product['image_url']:
                print(f"Скачиваю изображение {i+1}/{len(data['products'])}: {img_url}")
                local_path = download_image(img_url, id, session, save_dir)
                if local_path:
                    id += 1
                    paths.append(local_path)
                time.sleep(0.2)  # Задержка между загрузками
            product['local_image_path'] = paths
            product['downloaded'] = True

In [ ]:
download_images_for_products(result, SAVE_DIR)

save_results(result, filename='embroteka_imgs/embroteka.json')

print(f"\nИзображения сохранены в папку '{SAVE_DIR}'")

### Преобразование json в DataFrame

Для удобства работы данные, ранее собранные в json преобразуются в формат csv. Кроме того, все скачанные картинки пакуются в zip архив.

In [ ]:
## check max len of image_url
max_len = 0
for category_name, data in result.items():
    for i, product in enumerate(data['products']):
        max_len = max(max_len, len(product['image_url']))
max_len

8

In [ ]:
import pandas as pd
from itertools import chain

def flatten_product_data(product, max_imgs):
    """Преобразует данные продукта в плоскую структуру для DataFrame"""
    flat_data = {
        'name': product.get('name', ''),
        'url': product.get('url', ''),
        'price': product.get('price', ''),
        'description': product.get('description', ''),
        'category': product.get('category', []),
    }

    # Добавляем характеристики
    characteristics = product.get('characteristics', {})
    for key, value in characteristics.items():
        flat_data[key] = value

    # Добавляем информацию о изображениях
    images = product.get('image_url', [])
    if len(images) > max_imgs:
        print('!!! Слишком мало колонок для картинок (5)')
    for i, img_url in enumerate(images[:max_imgs]):  # Берем первые max_imgs изображений
        flat_data[f'image_url_{i+1}'] = img_url

    local_images = product.get('local_image_path', [])
    if len(local_images) != len(images):
        print('!!! Число путей к картинкам не равно числу картинок')
    for i, img_path in enumerate(local_images[:max_imgs]):
        flat_data[f'local_image_path_{i+1}'] = img_path

    return flat_data

def result_to_dataframe(result, max_imgs):
    """Преобразует результат парсинга в DataFrame"""
    rows = {}

    for category_name, category_data in result.items():
        for product in category_data['products']:
            # Добавляем категорию в данные продукта
            product['category'] = [category_name]
            # Преобразуем продукт в плоскую структуру
            flat_product = flatten_product_data(product, max_imgs)
            if product['url'] in rows:
                rows[product['url']]['category'].append(category_name)
                for path in product['local_image_path']:
                    if os.path.exists(path):
                        os.remove(path)
            else:
                rows[product['url']] = flat_product

    # Создаем DataFrame
    df = pd.DataFrame(rows.values())
    return df

In [ ]:
df = result_to_dataframe(result, max_len)
save_dataframe(df, base_filename='embroteka_imgs/embroteka.csv')
save_dataframe(df, base_filename='embroteka.csv')
df

Данные сохранены в CSV: embroteka_imgs/embroteka.csv
Данные сохранены в CSV: embroteka.csv


,name,url,price,description,category,"Размер дизайна, мм",Количество стежков,Количество цветов,Форматы файлов,image_url_1,...,local_image_path_3,local_image_path_4,image_url_5,local_image_path_5,image_url_6,image_url_7,image_url_8,local_image_path_6,local_image_path_7,local_image_path_8
0,Baby Tom,https://embroteka.ru/baby-tom-16926,300,"16926, Baby Tom, , 300 ₽, 16926, , Том и Джерр...","[Астрология и Космос, Кино, мульты, аниме, игры]","120х65,100х55, 80х45","6404, 5007, 3827",5,"dst, exp, hus, jef, pes, sew, vp3, xxx",https://embroteka.ru/image/cache/catalog/digit...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Behind,https://embroteka.ru/behind,6002,"Дизайн для вышивальных машин ""Behind"" подойдёт...","[Астрология и Космос, Кино, мульты, аниме, игры]",241x281,39557,12,"dst, emb, pes, vp3",https://embroteka.ru/image/cache/catalog/digit...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Eren x Nike logo,https://embroteka.ru/erennike-logo,200,"Дизайн для вышивальных машин ""Eren x Nike logo...","[Астрология и Космос, Кино, мульты, аниме, игры]","148x60, 174x70, 199x80","11786, 14474, 17447",3,"dst, exp, hus, jef, pes, sew, vip, xxx",https://embroteka.ru/image/cache/catalog/digit...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hello Kitty nike,https://embroteka.ru/hello-kitty-nike,300,"Дизайн для вышивальных машин ""Hello Kitty nike...","[Астрология и Космос, Логотипы, гербы, шевроны]",110x58,6998,4,pes,https://embroteka.ru/image/cache/catalog/digit...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Nike x Kapibara Найк х Капибара,https://embroteka.ru/nike-kapibara,850,"Дизайн для вышивальных машин ""Nike x Kapibara ...","[Астрология и Космос, Логотипы, гербы, шевроны]",223x97,24225,6,"dst, exp, pes, sew, xxx",https://embroteka.ru/image/cache/catalog/digit...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10889,Хонда Сивик 6,https://embroteka.ru/honda-civic6,50,"Дизайн для вышивальных машин ""Хонда Сивик 6"" п...",[Транспорт],93x28,3906,1,jef,https://embroteka.ru/image/cache/catalog/digit...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10890,Штурвал и лодка,https://embroteka.ru/steering-wheel,250,"Дизайн для вышивальных машин ""Штурвал и лодка""...",[Транспорт],140x140,10537,1,dst,https://embroteka.ru/image/cache/catalog/digit...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10891,Эскаватор,https://embroteka.ru/eskavator,Бесплатно,"Дизайн для вышивальных машин ""Эскаватор"" подой...",[Транспорт],155x144,25561,6,vp3,https://embroteka.ru/image/cache/catalog/digit...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10892,ЯК-52 Самолет,https://embroteka.ru/yk-52,350,"Дизайн для вышивальных машин ""ЯК-52 Самолет "" ...",[Транспорт],76x76,6980,5,dst,https://embroteka.ru/image/cache/catalog/digit...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
!zip -r 'embroteka_imgs.zip' 'embroteka_imgs'

## Парсинг сайта royal-present.ru

### Создание датасета

Аналогично парсингу embroteka, здесь сначала находятся ссылки на категории, а потом для каждой категории алгоритм итерируется по всем страницам и собирает ссылки и  некоторую информацию о товарах.

In [ ]:
def get_categories_from_html_rp(html_content, base_url):
    soup = BeautifulSoup(html_content, 'html.parser')
    categories = []
    category_items = soup.find_all('div', class_='category-teaser')

    for item in category_items:
        link = item.find('a', class_='image-category-teaser')
        if not link:
            continue

        category_name = item.find('h4').get_text(strip=True)
        category_url = urljoin(base_url, link['href'])

        if category_name == 'ДОПОЛНИТЕЛЬНЫЕ УСЛУГИ' or category_name == 'Подарочные Карты':
            continue
        categories.append({
            'name': category_name,
            'url': category_url
        })

    return categories


def get_product_data_rp(all_data, category, soup, base_url):
    product_items = soup.select('.product-card-inner')

    for item in product_items:
        # Извлекаем название продукта
        name_element = item.select_one('h4 .go-product')
        name = name_element.get_text(strip=True) if name_element else None

        # Извлекаем URL продукта
        url_element = item.select_one('.go-product')
        relative_url = url_element['href'] if url_element else None
        url = urljoin(base_url, relative_url) if relative_url else None

        # Извлекаем цену
        price_element = item.select_one('compare-to-price span')
        if price_element is None:
            price_element = item.select_one('.text-warning')
        price = price_element.get_text(strip=True).replace('руб.', '').strip() if price_element else None

        if url in all_data:
            all_data[url]['category'].append(category)
        else:
            all_data[url] = {
                'name': name,
                'url': url,
                'price': price,
                'category': [category],
            }
    return all_data

def get_all_products_from_category_rp(all_data, category, session, use_cache=True):
    category_url = category['url']
    category_name = category['name']
    page_num = 1

    while True:
        print(f'    Обрабатываю страницу: {page_num}')
        page_url = f"{category_url}/page/{page_num}" if page_num > 1 else f"{category_url}/"

        # Пробуем загрузить из кэша
        html_content = None
        if use_cache:
            html_content = load_page(page_url)

        # Если нет в кэше, скачиваем
        sleep = False
        if not html_content:
            sleep = True
            try:
                response = session.get(page_url)
                response.raise_for_status()
                html_content = response.text
                save_page(page_url, html_content)
            except Exception as e:
                print(f"Ошибка при загрузке страницы {page_url}: {e}")
                break

        soup = BeautifulSoup(html_content, 'html.parser')
        all_data = get_product_data_rp(all_data, category_name, soup, category_url)

        # Проверяем наличие кнопки "Следующая страница"
        next_page = soup.select_one('.pagination .next')
        if (next_page and ('disabled' in next_page.get('class'))) or not soup.select_one('.pagination'):
            break

        page_num += 1
        if sleep:
            time.sleep(1)  # Задержка между запросами

    return all_data

def parse_royal_present(base_url, use_cache=True):
    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    })

    try:
        # Пробуем загрузить главную страницу из кэша
        html_content = None
        if use_cache:
            html_content = load_page(base_url)

        # Если нет в кэше, скачиваем
        if not html_content:
            response = session.get(base_url)
            response.raise_for_status()
            html_content = response.text
            save_page(base_url, html_content)

        # Получаем список всех категорий
        categories = get_categories_from_html_rp(html_content, base_url)
    except Exception as e:
        print(f"Ошибка при получении главной страницы: {e}")
        return None

    all_data = {}

    for category in categories:
        print(f"Обрабатываю категорию: {category['name']}")

        try:
            products = get_all_products_from_category_rp(all_data, category, session, use_cache)
            time.sleep(2)  # Задержка между категориями

        except Exception as e:
            print(f"Ошибка при обработке категории {category['name']}: {e}")
            continue

    return list(all_data.values())

In [ ]:
base_url = "https://royal-present.ru/"
use_cache = True  # Использовать сохраненные страницы если они есть

result = parse_royal_present(base_url, use_cache)

### Заполнение описаний и деталей для каждого продукта

Теперь для каждого товара парсится его страница, откуда собирается дополнительная информация.

In [ ]:
def parse_product_page_rp(product_url, session, base_url, use_cache=True):
    """Парсит страницу продукта и извлекает характеристики и описание"""
    # Пробуем загрузить из кэша
    html_content = None
    if use_cache:
        html_content = load_page(product_url)

    # Если нет в кэше, скачиваем
    if not html_content:
        try:
            response = session.get(product_url)
            response.raise_for_status()
            html_content = response.text
            save_page(product_url, html_content)
            # Добавляем задержку между запросами
            time.sleep(0.2)
        except Exception as e:
            print(f"Ошибка при загрузке страницы продукта {product_url}: {e}")
            return None

    soup = BeautifulSoup(html_content, 'html.parser')

    # Извлекаем изображения
    check_images = []

    # Главное изображение
    main_image = soup.select_one('.main-img-container img')
    if main_image and 'data-src' in main_image.attrs:
        image_url = main_image['data-src']
        if not image_url.startswith('data:image'):
            check_images.append(urljoin(base_url, image_url))

    # Галерея изображений
    gallery_items = soup.select('.quick-gallery link')
    for img in gallery_items:
        image_url = img.get('href')
        if image_url and not image_url.startswith('data:image'):
            check_images.append(urljoin(base_url, image_url))

    # Извлекаем описание (все содержимое body)
    description = ""
    start_marker = '<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN" "http://www.w3.org/TR/REC-html40/loose.dtd">'

    # Находим позицию начала описания
    start_pos = html_content.find(start_marker)
    if start_pos != -1:
        # Извлекаем все от маркера до конца body
        description_html = html_content[start_pos:]
        # Создаем новый soup для этой части
        desc_soup = BeautifulSoup(description_html, 'html.parser').select_one('body')
        # Получаем текст
        description = desc_soup.get_text(" ", strip=True)
        # Удаляем лишние пробелы
        description = " ".join(description.split())

    return {
        'image_url': check_images,  # Удаляем дубликаты
        'description': description
    }

def enrich_product_data_rp(product_data, session, base_url, use_cache=True):
    """Обогащает данные продукта информацией со страницы продукта"""
    print(f"Обрабатываю продукт: {product_data['name']}")

    product_info = parse_product_page_rp(product_data['url'], session, base_url, use_cache)
    if product_info:
        product_data.update(product_info)
        product_data['downloaded'] = False

    return product_data

def enrich_all_products_data_rp(data, base_url, use_cache=True):
    """Обогащает данные всех продуктов в результате"""
    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    })
    for i, product in enumerate(data):
        if 'description' in product:
            continue  # Пропускаем уже обработанные
        try:
            data[i] = enrich_product_data_rp(product, session, base_url, use_cache)

        except Exception as e:
            print(f"Ошибка при обработке продукта {product['name']}: {e}")
            continue

    return result

In [ ]:
BASE_URL = "https://royal-present.ru/"
SAVE_DIR = "royal_present_imgs"
os.makedirs(SAVE_DIR, exist_ok=True)

result = enrich_all_products_data_rp(result, BASE_URL, use_cache=True)

save_results(result, "royal_present_imgs/royal_present.json")

### Скачивание картинок

По ранее полученным ссылкам здесь скачиваются картинки. Так как картинок оказалось довольно много, то они были сохранены в несколько отдельных zip архивов.

In [ ]:
def download_images_for_products(result, save_dir):
    """Скачивает изображения для всех продуктов в датасете"""
    # Сессия для запросов
    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    })

    id = 0
    for i, product in enumerate(result):
        if 'downloaded' in product and product['downloaded']:
            id += len(product['local_image_path'])
            continue
        paths = []
        for img_url in product['image_url']:
            print(f"Скачиваю изображение {i+1}/{len(result)}: {img_url}")
            local_path = download_image(img_url, id, session, save_dir)
            if local_path:
                id += 1
                paths.append(local_path)
            time.sleep(0.2)  # Задержка между загрузками
        product['local_image_path'] = paths
        product['downloaded'] = True

In [ ]:
SAVE_DIR = "royal_present_imgs"
download_images_for_products(result, SAVE_DIR)

In [ ]:
save_results(result, filename='royal_present_imgs/royal_present.json')
!zip -r 'royal_present.zip' 'royal_present_imgs'

In [ ]:
SAVE_DIR = "royal_present_imgs_2"
os.makedirs(SAVE_DIR, exist_ok=True)
download_images_for_products(result, SAVE_DIR)

In [ ]:
save_results(result, filename='royal_present_imgs_2/royal_present.json')
!zip -r 'royal_present_2.zip' 'royal_present_imgs_2'

In [ ]:
BASE_URL = "https://royal-present.ru/"
SAVE_DIR = "royal_present_imgs_3"
os.makedirs(SAVE_DIR, exist_ok=True)
download_images_for_products(result, SAVE_DIR)

In [ ]:
save_results(result, filename='royal_present_imgs_3/royal_present.json')
!zip -r 'royal_present_3.zip' 'royal_present_imgs_3'

In [ ]:
BASE_URL = "https://royal-present.ru/"
SAVE_DIR = "royal_present_imgs_4"
os.makedirs(SAVE_DIR, exist_ok=True)
download_images_for_products(result, SAVE_DIR)
save_results(result, filename='royal_present_imgs_4/royal_present.json')

In [ ]:
save_results(result, filename='royal_present_imgs_4/royal_present.json')
!zip -r 'royal_present_4.zip' 'royal_present_imgs_4'

### Преобразование json в DataFrame

Для удобства работы созданный ранее json с метаданными здесь преобразуется в pd.DataFrame.

In [ ]:
## check max len of image_url
max_len = 0
for i, product in enumerate(result):
    max_len = max(max_len, len(product['image_url']))
max_len

28

In [ ]:
import pandas as pd
from itertools import chain

def flatten_product_data(product, max_imgs):
    """Преобразует данные продукта в плоскую структуру для DataFrame"""
    flat_data = {
        'name': product.get('name', ''),
        'url': product.get('url', ''),
        'price': product.get('price', ''),
        'description': product.get('description', ''),
        'category': product.get('category', ''),
    }

    # Добавляем информацию о изображениях
    images = product.get('image_url', [])
    if len(images) > max_imgs:
        print('!!! Слишком мало колонок для картинок (5)')
    for i, img_url in enumerate(images[:max_imgs]):  # Берем первые max_imgs изображений
        flat_data[f'image_url_{i+1}'] = img_url

    local_images = product.get('local_image_path', [])
    if len(local_images) != len(images):
        print('!!! Число путей к картинкам не равно числу картинок')
    for i, img_path in enumerate(local_images[:max_imgs]):
        flat_data[f'local_image_path_{i+1}'] = img_path

    return flat_data

def result_to_dataframe(result, max_imgs):
    """Преобразует результат парсинга в DataFrame"""
    rows = []
    for product in result:
        # Преобразуем продукт в плоскую структуру
        flat_product = flatten_product_data(product, max_imgs)
        rows.append(flat_product)

    # Создаем DataFrame
    df = pd.DataFrame(rows)
    return df

In [ ]:
df = result_to_dataframe(result, max_len)
save_dataframe(df, base_filename='royal_present.csv')
df

Данные сохранены в CSV: royal_present.csv


,name,url,price,description,category,image_url_1,image_url_2,local_image_path_1,local_image_path_2,image_url_3,...,image_url_25,image_url_26,image_url_27,local_image_path_23,local_image_path_24,local_image_path_25,local_image_path_26,local_image_path_27,image_url_28,local_image_path_28
0,Рогатый Дракон дизайн вышивки - 3 размера,https://royal-present.ru/Rogaty-Drakon-dizayn-...,350,"Размер: 86.5x98.4 mm (3.41x3.87 ""), Стежки: 11...","[Самые популярные дизайны, Зимние дизайны, Зна...",https://cdn.royal-present.ru/ruproduct-Rogaty-...,https://cdn.royal-present.ru/ruproduct-Rogaty-...,royal_present_imgs/0.jpg,royal_present_imgs/1.jpg,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Гжель Петух - 4 размер,https://royal-present.ru/gzhiel-pietukh-dizain...,600,Подробнее о лицензии Размер: 69.7x98.0 mm (2.7...,"[Самые популярные дизайны, Новые дизайны, Праз...",https://cdn.royal-present.ru/ruproduct-gzhiel-...,https://cdn.royal-present.ru/ruproduct-gzhiel-...,royal_present_imgs/2.jpg,royal_present_imgs/3.jpg,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Восточный Павлин на цветущей ветке - 3 размера,https://royal-present.ru/Vostochny-Pavlin-na-t...,800,Подробнее о лицензиях Роскошный павлин станет ...,"[Самые популярные дизайны, Восточные и Азиатск...",https://cdn.royal-present.ru/ruproduct-Vostoch...,https://cdn.royal-present.ru/ruproduct-Vostoch...,royal_present_imgs/4.jpg,royal_present_imgs/5.jpg,https://cdn.royal-present.ru/ruproduct-Vostoch...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Дизайн машинной вышивки Васильки - 2 размера,https://royal-present.ru/Dizayn-mashinnoy-vysh...,400,"Размер: 67.0x178.4 mm (2.64x7.02 ""), Стежки: 7...","[Самые популярные дизайны, Цветы и Растения, П...",https://cdn.royal-present.ru/ruproduct-Dizayn-...,https://cdn.royal-present.ru/ruproduct-Dizayn-...,royal_present_imgs/10.jpg,royal_present_imgs/11.jpg,https://cdn.royal-present.ru/ruproduct-Dizayn-...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Дизайн машинной вышивки Вишенки ришелье салфет...,https://royal-present.ru/Dizayn-mashinnoy-vysh...,450,"Размер: 194.7x199.4 mm (7.67x7.85 ""), Стежки: ...","[Самые популярные дизайны, Кружевные дизайны |...",https://cdn.royal-present.ru/ruproduct-Dizayn-...,https://cdn.royal-present.ru/ruproduct-Dizayn-...,royal_present_imgs/14.jpg,royal_present_imgs/15.jpg,https://cdn.royal-present.ru/ruproduct-Dizayn-...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5132,Дизайн машинной вышивки Камин с носками,https://royal-present.ru/Dizayn-mashinnoy-vysh...,270,Дизайн для пялец 100х100 мм; 130х180 мм; 140х2...,[RedWork - Ред Ворк],https://cdn.royal-present.ru/ruproduct-Dizayn-...,https://cdn.royal-present.ru/ruproduct-Dizayn-...,royal_present_imgs_4/23222.jpg,royal_present_imgs_4/23223.jpg,https://cdn.royal-present.ru/ruproduct-Dizayn-...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5133,Новогодний Камин Red Work Дизайн для машинной ...,https://royal-present.ru/Novogodny-Kamin-Red-W...,270,Дизайн для пялец 100х100 мм; 130х180 мм; 140х2...,[RedWork - Ред Ворк],https://cdn.royal-present.ru/ruproduct-Novogod...,https://cdn.royal-present.ru/ruproduct-Novogod...,royal_present_imgs_4/23228.jpg,royal_present_imgs_4/23229.jpg,https://cdn.royal-present.ru/ruproduct-Novogod...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5134,Рождественское настроение с Дедом Морозом Диза...,https://royal-present.ru/Rozhdestvenskoe-nastr...,270,"Для пялец 200х200 мм Форматы: .pes, .pec, .hus...",[RedWork - Ред Ворк],https://cdn.royal-present.ru/ruproduct-Rozhdes...,https://cdn.royal-present.ru/ruproduct-Rozhdes...,royal_present_imgs_4/23234.jpg,royal_present_imgs_4/23235.jpg,https://cdn.royal-present.ru/ruproduct-Rozhdes...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5135,Дизайн для машинной вышивки Рождественское нас...,https://royal-present.ru/Dizayn-dlya-mashinnoy...,270,"Для пялец 200х200 мм Форматы: .pes, .pec, .hus...",[RedWork - Ред Ворк],https

In [ ]:
save_results(result, filename='royal_present_imgs/royal_present.json')
save_results(result, filename='royal_present.json')
!zip -r 'royal_present.zip' 'royal_present_imgs'